In [1]:
#r "nuget:Google.OrTools"

using Google.OrTools.LinearSolver;
using System.Linq; 

Installed Packages Google.OrTools, 9.3.10497

In [1]:
//We prepare a function to call the solver solve. Code reuse!
public static void SolveAndPrint(this Solver solver, int nItems, double[] weights)
{
    var resultStatus = solver.Solve();

    // Check that the problem has an optimal solution.
    if (resultStatus != Solver.ResultStatus.OPTIMAL)
    {
        Console.WriteLine("The problem does not have an optimal solution!");
        return;
    }

    Console.WriteLine("Problem solved in " + solver.WallTime() + " milliseconds");

    // The objective value of the solution.
    Console.WriteLine("Optimal objective value = " + solver.Objective().Value());

    display(solver.variables().Select(a => new { Name  = a.Name(), Value = a.SolutionValue() }));
    
    // displaying solution weight (custom to this problem)
    double totalWeight = 0.0; 
    for(int i = 0; i < nItems; i++)
        totalWeight += solver.variables()[i].SolutionValue() * weights[i]; 

    display(new { Name = "Total Weight", Weight = totalWeight});
}

# Applications of linear programming

## Knapsack Problem 

The knapsack problem defines a bag that was a maximal weight of $W$, which we can fill from a set of items each with a weight of $w_i$ and a value of $v_i$. Typically, the problem is defined with an $x_i \in \{0,1\}$ (binary) variable set to either 0 or 1 knapsack where each item is either taken or not. 

Here we are going to allow fractional parts of the items to be taken so that we can solve it as a linear problem (also known as a [linear relaxation](https://en.wikipedia.org/wiki/Linear_programming_relaxation)), allowing $x_i \in [0,1]$. 

### linear relaxation 

The relaxed linear model of the knapsack problem is formulated as follows: 

$$
\begin{aligned}
\text{Obj:} \max \sum ^{n}_{i=1}\left(x_{i} v_{i}\right)
\\st: \qquad 0 \leq x_{i} \leq 1
\\ \qquad \quad \sum ^{n}_{i=1} x_{i}w_{i} \leq W
\end{aligned}
$$

In [1]:
//Array of items, weights and totals
int nItems = 10; 
double maxWeight = 220; 
double[] weights = {31, 27, 12, 39,  2, 69, 66, 29, 45, 58};
double[] values =  {24, 27, 26, 15, 19, 33, 30, 28, 65, 42};

The solver is already defined and ```SolveAndPrint(solver);``` will be automatically called when you click run. 

Of note is that we can use the MakeArray functions of the solver to create a set of variables all in one go. 

In [1]:
Solver solver = Solver.CreateSolver("CLP_LINEAR_PROGRAMMING");
Variable[] Items = solver.MakeNumVarArray(10, 0.0, 1.0, "Items");

// Maximize revenue 
Objective objective = solver.Objective();
for(int i = 0; i < nItems; i++) objective.SetCoefficient(Items[i], values[i]);
objective.SetMaximization();

// Weight limit
Constraint c0 = solver.MakeConstraint(0, maxWeight);
for(int i = 0; i < nItems; i++) c0.SetCoefficient(Items[i], weights[i]);

SolveAndPrint(solver, nItems, weights);

Optimal objective value = 238,6521739130435


index,Name,Value
0,Items0,1
1,Items1,1
2,Items2,1
3,Items3,0
4,Items4,1
5,Items5,0.23188405797101463
6,Items6,0
7,Items7,1
8,Items8,0.9999999999999999
9,Items9,1


Name,Weight
Total Weight,220


Notice Item5 and Item8 have partial values that are not 1.0 or 0. 

### 0/1 knapsack 

When we want to solve the original problem were no fractions of the items can be taken, we have to indroduce binary variables, this makes the problem non-linear and we need to solve a MILP problem instead of an LP problem. Adding binary variables to problems allows for many advanced formulations, but come at a high computational cost. Add too many binary variables and your optimizations problems will take too long to solve. 

$$x_i \in \mathbb{Z},\quad x_i \in \{0,1\}$$

In the Google OR tools, we can either use the MakeBoolVarArray or the MakeIntVarArray to create the binary variables we want.

In [1]:
Solver milp_solver = Solver.CreateSolver("CBC_MIXED_INTEGER_PROGRAMMING");

Variable[] Items = milp_solver.MakeBoolVarArray(10, "Items");

// Maximize revenue 
Objective objective = milp_solver.Objective();
for(int i = 0; i < nItems; i++) objective.SetCoefficient(Items[i], values[i]);
objective.SetMaximization();

// Weight limit
Constraint c0 = milp_solver.MakeConstraint(0, maxWeight);
for(int i = 0; i < nItems; i++) c0.SetCoefficient(Items[i], weights[i]);

SolveAndPrint(milp_solver, nItems, weights);

Optimal objective value = 231


index,Name,Value
0,Items0,1
1,Items1,1
2,Items2,1
3,Items3,0
4,Items4,1
5,Items5,0
6,Items6,0
7,Items7,1
8,Items8,1
9,Items9,1


Name,Weight
Total Weight,204


The objective value of the problem is lower because an integer constraint is more restrictive than a linear one and we no longer have the partial values for the items in the knapsack, and item5 is no longer part of the solution. You may also notice that there is a remaining weight available in the knapsack. 

Note that if you try to create the Bool or Int array in the relaxed version above, you will still get the same result as before, because the solver for that problem was defined as a ```"CLP_LINEAR_PROGRAMMING"``` solver, whereas this code block has a solver defined with ```"CBC_MIXED_INTEGER_PROGRAMMING"```. 

## Notes
There is actually a specialised solver for the [knapsack problem](https://developers.google.com/optimization/bin/knapsack) in the OR-Tools which also covers multi-dimensional knapsacks. Here I felt it was useful to contrast the linear and mixed integer solutions.